# TFM Suministro Global - CREAR RELACION -NECESITA_DE - de dependencia  entre materiales:
## Materiales Refinados dependen de Materiales Brutos y y pueden generarse al extraerse/procesarse  otros materiales distintos al actual.  
### Ejemplo: el Aluminio depende de la Bauxita, y  se reflejará con una relación adicional   
### _AluminioB-[NECESITA_DE]-> BauxitaB)_

##    > CREAR relación  MaterialREF--[NECESITA_DE] -->  MaterialB
##    > Si aplica, CREAR relación adicional  MaterialB--[NECESITA_DE] -->  MaterialB
##    > Si aplica, CREAR relación adicional  MaterialREF--[NECESITA_DE] -->  MaterialB
##    > Si aplica, CREAR nodo nuevo con propiedad específica: MaterialB {nuevo: "SI"}

## _Neo4j - py2neo_

**OBSERVACIONES**  
Ya creados los nodos ORIGEN y DESTINO
Se carga fichero de adicionales para añadir relaciones adicionales para materiales específicos
Va a añadir 3 nodos específicos de otros materiales que no están en el estudio pero que completan la cadena de suministro para el caso de los sulfuros ("Sulphur"), que depende en parte de la obtención de Gas Natural, o combustibles fósiles

## Partimos de la existencia, en paralelo a esta ejecución, de una instancia levantada y accesible de Neo4j  
### La primera vez que se quiera usar _py2neo_ erá  necesario asegurar su instalación:  _!pip install py2neo_

In [1]:
##!pip install py2neo 

In [2]:
#importar la librería para poder hacer uso de sus funcionalidades y objetos
from py2neo import *

In [3]:
#acceder a la instancia levantada de Neo4j - en nuestro caso instancia Docker - asignamos credenciales de autentificación -
graph_test01 = Graph(auth=('neo4j', 's3cr3t'))

In [4]:
#que es el objeto - graph_test01
graph_test01

Graph('bolt://localhost:7687')

In [5]:
#que es el objeto - tx
#tx = graph_test01.begin()

In [6]:
#tx

In [7]:
#query de conexión de prueba sin errores
# verificar existencia de NODOS previos creados
#--- cuenta los nodos con diferentes tipos de etiqueta y los une en una única salida, 1 fila por cada tipo (name, cnt)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[3] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
"
graph_test01.run(queryPRB).to_data_frame()

,name,Ntotal
0,MaterialB,79
1,MaterialREF,80
2,Pais,116
3,GrupoAplicacion,88
4,SectorAplicacion,25


## CARGAR DATOS

In [8]:
import numpy as np    #datos y operativa álgegra y matrices de n dimensiones (arrays)
import pandas as pd   #carga dataset y manipulación de datos  y dataframes

In [9]:
#DATOS: Fichero completo de MATERIAS PRIMAS CRÍTICAS en el suministro GLOBAL - Atendiendo al alcance del estudio - EU CRM 2020
#FICHERO: GLMateriales_cargar.csv
#CONTENIDO: Materiales 
#USO: Crear los NODOS de tipo "MaterialB" - Material bruto y "MaterialREF" - Material Refinado) 
## -- propiedades de cada material: nombre, abreviatura, familia y subfamilia 
## -- los datos de suministro -posteriores- fijarán las relaciones entre materiales, países proveedores y usos del material 
rutaFichero =  "https://raw.githubusercontent.com/JuanKFD/UNIR_TFM/d10cd756788ce5e4b93f6b5c7ca4d40cb4aed297/GLMateriales_cargar.csv" 
dfMateriales = pd.read_csv(rutaFichero, sep=';')


In [10]:
dfMateriales.head()

,Material,Mat_Abrev,Mat_Familia,Mat_Subfamilia
0,Aluminium,Al,General,Other non-ferrous metals
1,Antimony,Sb,General,Other non-ferrous metals
2,Arsenic,As,General,Other non-ferrous metals
3,Baryte,GBrt,General,Industrial & constr. minerals
4,Bauxite,Bx,General,Other non-ferrous metals


In [11]:
dfMateriales.tail()

,Material,Mat_Abrev,Mat_Familia,Mat_Subfamilia
73,Tungsten,W,General,Iron and ferro-alloy metals
74,Vanadium,V,General,Iron and ferro-alloy metals
75,Yttrium,Y,Rare earths,HREEs-Heavy
76,Zinc,Zn,General,Other non-ferrous metals
77,Zirconium,Zr,General,Other non-ferrous metals


In [14]:
dfMateriales.shape

(78, 4)

In [14]:
nFilas = dfMateriales.shape[0]

In [15]:
nFilas

78

# PREVISUALIZACION DE DATOS - MATERIALES
a = Node("Material", name=dfMateriales.Material[i])

In [16]:
for i in range(0, nFilas):
    print(dfMateriales.Material[i])
    print(dfMateriales.Mat_Abrev[i])
    print(dfMateriales.Mat_Familia[i])
    print(dfMateriales.Mat_Subfamilia[i])

Aluminium
Al
General
Other non-ferrous metals
Antimony
Sb
General
Other non-ferrous metals
Arsenic
As
General
Other non-ferrous metals
Baryte
GBrt
General
Industrial & constr. minerals
Bauxite
Bx
General
Other non-ferrous metals
Bentonite
GBn
General
Industrial & constr. minerals
Beryllium
Be
General
Other non-ferrous metals
Bismuth
Bi
General
Other non-ferrous metals
Borate
Gbo
General
Industrial & constr. minerals
Cadmium
Cd
General
Other non-ferrous metals
Cerium
Ce
Rare earths
LREEs-Light
Chromium
Cr
General
Iron and ferro-alloy metals
Cobalt
Co
General
Iron and ferro-alloy metals
Coking coal
GCc
General
Bio and other materials
Copper
Cu
General
Other non-ferrous metals
Diatomite
Di
General
Industrial & constr. minerals
Dysprosium
Dy
Rare earths
HREEs-Heavy
Erbium
Er
Rare earths
HREEs-Heavy
Europium
Eu
Rare earths
HREEs-Heavy
Feldspar
Fsp
General
Industrial & constr. minerals
Fluorspar
Fl
General
Industrial & constr. minerals
Gadolinium
Gd
Rare earths
HREEs-Heavy
Gallium
Ga
General

In [21]:
dfMateriales.columns

Index(['Material', 'Mat_Abrev', 'Mat_Familia', 'Mat_Subfamilia'], dtype='object')

# CREAR RELACION _-NECESITA_DE-_ entre nodos  :MaterialB y :MaterialREF

Consulta patrón CQL-Neo4j para crear la relación buscada
MATCH
(a:MaterialREF),
(b:MaterialB)

WHERE a.name = dfMateriales.Material[i] AND b.name = dfMateriales.Material[i]
CREATE (a)-[r:NECESITA_DE {SHR_PaisMat: 0.nn, t_PaisMat: m.mm }]->(b)
RETURN type(r)

In [18]:
#  Crear RELACIONES de tipo :NECESITA_DE
#  Nodos origen y destino: MATERIALES -- "MaterialB" bruto -->  "MaterialREF" refinado
#
## PREVISAULIZAR QUERYs antes de la ejecución
# 
TipoNodoA = " :MaterialB" 
TipoNodoB = " :MaterialREF" 
TipoRelacion = " :NECESITA_DE"
inicio = 0
nFilas = dfMateriales.shape[0]
#
for i in range(inicio, nFilas):
        strQry = "MATCH (a"                                    \
            + TipoNodoA                                        \
            + "), (b " + TipoNodoB                            \
            + ") WHERE a.name = '"                             \
            +  dfMateriales.Material[i]                       \
            + "'"                                             \
            + " AND b.name = '"                               \
            +  dfMateriales.Material[i]                      \
            + "_REF'"                                         \
            + " CREATE (b)-[r"                               \
            + TipoRelacion                                    \
            + "]->(a)"                                        \
            + " RETURN type(r)"                                 
        print("QRY [" + str(i) + "]> "+ strQry)

# querys  visualizadas


QRY [0]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Aluminium' AND b.name = 'Aluminium_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
QRY [1]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Antimony' AND b.name = 'Antimony_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
QRY [2]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Arsenic' AND b.name = 'Arsenic_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
QRY [3]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Baryte' AND b.name = 'Baryte_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
QRY [4]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Bauxite' AND b.name = 'Bauxite_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
QRY [5]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Bentonite' AND b.name = 'Bentonite_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
QRY [6]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Beryllium' AND b.name = 'Berylliu

In [20]:
#verificar que se mantiene la conexión creada
graph_test01

Graph('bolt://localhost:7687')

In [24]:
# Crear en BD Neo4J todas  las relaciones NECESITA_DE
TipoNodoA = " :MaterialB" 
TipoNodoB = " :MaterialREF" 
TipoRelacion = " :NECESITA_DE"
inicio = 0
nFilas = dfMateriales.shape[0]
#
for i in range(inicio, nFilas):
    strQry = "MATCH (a"                                    \
            + TipoNodoA                                        \
            + "), (b " + TipoNodoB                            \
            + ") WHERE a.name = '"                             \
            +  dfMateriales.Material[i]                       \
            + "'"                                             \
            + " AND b.name = '"                               \
            +  dfMateriales.Material[i]                      \
            + "_REF'"                                         \
            + " CREATE (b)-[r"                               \
            + TipoRelacion                                    \
            + "]->(a)"                                        \
            + " RETURN type(r)"                                 
#   lanzar query CQL en la BD
    graph_test01.run(strQry)
#   Visualizar la Query lanzada
    print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
#--fin for---
#
# relaciones creadas

EJECUTADA QRY [0]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Aluminium' AND b.name = 'Aluminium_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUTADA QRY [1]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Antimony' AND b.name = 'Antimony_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUTADA QRY [2]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Arsenic' AND b.name = 'Arsenic_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUTADA QRY [3]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Baryte' AND b.name = 'Baryte_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUTADA QRY [4]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Bauxite' AND b.name = 'Bauxite_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUTADA QRY [5]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Bentonite' AND b.name = 'Bentonite_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUTADA QRY [6]> MATCH (a :MaterialB

EJECUTADA QRY [58]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Samarium' AND b.name = 'Samarium_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUTADA QRY [59]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Sapele wood' AND b.name = 'Sapele wood_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUTADA QRY [60]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Scandium' AND b.name = 'Scandium_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUTADA QRY [61]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Selenium' AND b.name = 'Selenium_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUTADA QRY [62]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Silica sand' AND b.name = 'Silica sand_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUTADA QRY [63]> MATCH (a :MaterialB), (b  :MaterialREF) WHERE a.name = 'Silicon metal' AND b.name = 'Silicon metal_REF' CREATE (b)-[r :NECESITA_DE]->(a) RETURN type(r)
EJECUT

In [25]:
#validamos que todas las RELACIONES se han creado (hasta este paso)
queryPRB= "MATCH n=()-[r:INCLUIDO_EN]->()              \
RETURN type(r) as TipoRelacion, count (r) as Ntotal    \
UNION MATCH m=()-[p:USADO_EN]->()                      \
RETURN type(p) as TipoRelacion, count (p) as Ntotal    \
UNION MATCH m=()-[p:REFINA]->()                        \
RETURN type(p) as TipoRelacion, count (p) as Ntotal    \
UNION MATCH m=()-[p:EXTRAE]->()                        \
RETURN type(p) as TipoRelacion, count (p) as Ntotal    \
UNION MATCH m=()-[p:NECESITA_DE]->()                   \
RETURN type(p) as TipoRelacion, count (p) as Ntotal" 

graph_test01.run(queryPRB).to_data_frame()

,TipoRelacion,Ntotal
0,INCLUIDO_EN,442
1,USADO_EN,442
2,REFINA,475
3,EXTRAE,699
4,NECESITA_DE,100


# CREAR RELACION _-NECESITA_DE-_ para lista ADICIONAL de materiales entre   :MaterialB y  Material de Origen (:MaterialREF o :MaterialB) cuyo  tipo es determinado según el número de etapa de su obtención (EtapaBase con valores 1  para REF ó 2 para B ). En el caso de Material origen no especificado en las tablas de dataset del estudio de partida, se añaden algunos nuevos por completitud del grafo (material origen comenzando por "N-" será creado).
## 1- Carga del fichero de dependencias adicionales de materiales
## 2- Crear nodos de materiales adicionales (si aplica - "comienzo por "N-")
## 3- Crear relación NECESITA_DE entre materiales adicionales BRUTOS y material de origen 


## 1- Carga del fichero de dependencias adicionales de materiales

In [27]:
#DATOS: Fichero con MATERIALES y MATERIALES ORIGEN PRIMAS de los que NECESITA adicionalmente
#       según elsuministro GLOBAL - Atendiendo al alcance del estudio - EU CRM 2020
#FICHERO: GLAddMatDepend_Cargar.csv
#CONTENIDO: relación de Materiales y Materiales Origen para fijar relaciones NECESITA_DE adicionales a las estándar 
#USO: Crear relación "NECESITA_DE" entre materiales adicionales: columna "Material" y columna "MaterialOrg" 
rutaFichero =  "https://raw.githubusercontent.com/JuanKFD/UNIR_TFM/main/GLAddMatDepend_Cargar.csv" 
dfMaterialADIC = pd.read_csv(rutaFichero, sep=';')
# NOTA.- el fichero admite 3 columnas adicionales para usos futuros

In [29]:
dfMaterialADIC.head(22)

,Material,MaterialOrg,EtapaBase,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Aluminium,Bauxite,1,NaN,NaN,NaN
1,Arsenic,Copper,1,NaN,NaN,NaN
2,Arsenic,Zinc,1,NaN,NaN,NaN
3,Cadmium,Zinc,1,NaN,NaN,NaN
4,Germanium,Zinc,1,NaN,NaN,NaN
5,Hafnium,Zirconium,1,NaN,NaN,NaN
6,Selenium,Copper,1,NaN,NaN,NaN
7,Tellurium,Copper,1,NaN,NaN,NaN
8,Iridium,Nickel,1,NaN,NaN,NaN
9,Palladium,Nickel,1,NaN,NaN,NaN


In [30]:
dfMaterialADIC.tail()

,Material,MaterialOrg,EtapaBase,Unnamed: 3,Unnamed: 4,Unnamed: 5
17,Ruthenium,Copper,1,NaN,NaN,NaN
18,Sulphur,Coking coal,2,NaN,NaN,NaN
19,Sulphur,N-Fossil Fuels,2,NaN,NaN,NaN
20,Sulphur,N-Natural gas,2,NaN,NaN,NaN
21,Sulphur,N-Sulphur,1,NaN,NaN,NaN


In [31]:
dfMaterialADIC.shape

(22, 6)

In [32]:
dfMaterialADIC.columns

Index(['Material', 'MaterialOrg', 'EtapaBase', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5'],
      dtype='object')

In [33]:
# REGLAS DEL PROCESO:
#     Si EtapaBase == '1', 
#             entonces la relación Material es con  MaterialOrg de TipoNODOAsignar= ":MaterialB" y SUFIJOorg = ""    
#     Si EtapaBase == '2', 
#             entonces la relación Material es con  MaterialOrg de TipoNODOAsignar= ":MaterialREF" y SUFIJOorg = "_REF"
#     Si MaterialOrg[0:2] == "N-",  ---- material NUEVO inexistente y hay que crearlo ----
#             entonces 
#                  CREAR NODO del tipo TipoNODOAsignar y  name MaterialOrg+SUFIJOOrg  y con propiedad {nuevo: "SI"}
#
#     CREAR RELACION "NECESITA_DE" entre MaterialB name= Material  y  :TipoNODOAsignar con name MaterialOrg+SUFIJOOrg  
##
# Crear en BD Neo4J todas  las relaciones NECESITA_DE
TipoNodoA = " :MaterialB" 
TipoNodoB = " :MaterialREF" 
TipoRelacion = " :NECESITA_DE"
PrefijoNUEVOMat = 'N-'
SUFIJOorg = ""
inicioADIC = 0
nFilasADIC = dfMaterialADIC.shape[0]
#
for i in range(inicio, nFilasADIC):
    TipoNODOAsignar=""
    SUFIJOorg = ""
    if dfMaterialADIC.EtapaBase[i] == 1:                                        #caso de material origen Bruto
        TipoNODOAsignar = TipoNodoA
        SUFIJOorg = ""
    elif dfMaterialADIC.EtapaBase[i] == 2:                                   #caso de material origen Refinado
        TipoNODOAsignar = TipoNodoB
        SUFIJOorg = "_REF"
    else:                                                       #caso -error- que NO debería darse 
        TipoNODOAsignar = ":tipo_error"
        SUFIJOorg = "_err"
# control si hay que crear un nodo de material NUEVO
    if dfMaterialADIC.MaterialOrg[i][0:2] == PrefijoNUEVOMat:   #caso de crear nuevo nodo material
        strQryCRND = "CREATE (m "                       \
        + TipoNODOAsignar                                \
        + " {name: '"                                    \
        +  dfMaterialADIC.MaterialOrg[i]+SUFIJOorg          \
        +  "', nuevo: '"          \
        + "SI"                                           \
        +"'})"
        print("EJEC. CREA NODO [" + str(i) + "]> "+ strQryCRND)
        
# se haya creado nodo origen o se usen nodos existentes, crear la relación NECESITA_DE adicional entre los nodos
    strQry = "MATCH (a"                                    \
            + TipoNodoA                                     \
            + "), (b " + TipoNODOAsignar                     \
            + ") WHERE a.name = '"                          \
            +  dfMaterialADIC.Material[i]                    \
            + "'"                                           \
            + " AND b.name = '"                             \
            +  dfMaterialADIC.MaterialOrg[i]+SUFIJOorg         \
            + "' CREATE (a)-[r"                               \
            + TipoRelacion                                   \
            + "]->(b)"                                        \
            + " RETURN type(r)"                                 
# Visualizar la Query lanzada
    print("QRY [" + str(i) + "]> "+ strQry)
#
# queries preparadas y visualizadas para control

QRY [0]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Aluminium' AND b.name = 'Bauxite' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
QRY [1]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Arsenic' AND b.name = 'Copper' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
QRY [2]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Arsenic' AND b.name = 'Zinc' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
QRY [3]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Cadmium' AND b.name = 'Zinc' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
QRY [4]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Germanium' AND b.name = 'Zinc' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
QRY [5]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Hafnium' AND b.name = 'Zirconium' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
QRY [6]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Selenium' AND b.name = 'Copper' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
QRY

In [86]:
# REGLAS DEL PROCESO:
#     Si EtapaBase == '1', 
#             entonces la relación Material es con  MaterialOrg de TipoNODOAsignar= ":MaterialB" y SUFIJOorg = ""    
#     Si EtapaBase == '2', 
#             entonces la relación Material es con  MaterialOrg de TipoNODOAsignar= ":MaterialREF" y SUFIJOorg = "_REF"
#     Si MaterialOrg[0:2] == "N-",  ---- material NUEVO inexistente y hay que crearlo ----
#             entonces 
#                  CREAR NODO del tipo TipoNODOAsignar y  name MaterialOrg+SUFIJOOrg  y con propiedad {nuevo: "SI"}
#
#     CREAR RELACION "NECESITA_DE" entre MaterialB name= Material  y  :TipoNODOAsignar con name MaterialOrg+SUFIJOOrg  
##
# Crear en BD Neo4J todas  las relaciones NECESITA_DE
TipoNodoA = " :MaterialB" 
TipoNodoB = " :MaterialREF" 
TipoRelacion = " :NECESITA_DE"
PrefijoNUEVOMat = 'N-'
SUFIJOorg = ""
inicioADIC = 0
nFilasADIC = dfMaterialADIC.shape[0]
#
for i in range(inicio, nFilasADIC):
    TipoNODOAsignar=""
    SUFIJOorg = ""
    if dfMaterialADIC.EtapaBase[i] == 1:                                        #caso de material origen Bruto
        TipoNODOAsignar = TipoNodoA
        SUFIJOorg = ""
    elif dfMaterialADIC.EtapaBase[i] == 2:                                   #caso de material origen Refinado
        TipoNODOAsignar = TipoNodoB
        SUFIJOorg = "_REF"
    else:                                                       #caso -error- que NO debería darse 
        TipoNODOAsignar = ":tipo_error"
        SUFIJOorg = "_err"
# control si hay que crear un nodo de material NUEVO
    if dfMaterialADIC.MaterialOrg[i][0:2] == PrefijoNUEVOMat:   #caso de crear nuevo nodo material
        strQryCRND = "CREATE (m "                       \
        + TipoNODOAsignar                                \
        + " {name: '"                                    \
        +  dfMaterialADIC.MaterialOrg[i]+SUFIJOorg          \
        +  "', nuevo: '"          \
        + "SI"                                           \
        +"'})"
        print("EJEC. CREA NODO [" + str(i) + "]> "+ strQryCRND)
# lanzar query CQL de crear NODO en la BD si es necesario por las condiciones del if
        graph_test01.run(strQryCRND)
        
# se haya creado nodo origen o se usen nodos existentes, crear la relación NECESITA_DE adicional entre los nodos
    strQry = "MATCH (a"                                    \
            + TipoNodoA                                     \
            + "), (b " + TipoNODOAsignar                     \
            + ") WHERE a.name = '"                          \
            +  dfMaterialADIC.Material[i]                    \
            + "'"                                           \
            + " AND b.name = '"                             \
            +  dfMaterialADIC.MaterialOrg[i]+SUFIJOorg         \
            + "' CREATE (a)-[r"                               \
            + TipoRelacion                                   \
            + "]->(b)"                                        \
            + " RETURN type(r)"                                 
# lanzar query CQL en la BD para CREAR LA RELACION
    graph_test01.run(strQry)
# Visualizar la Query lanzada
    print("EJEC. QRY [" + str(i) + "]> "+ strQry)
#
# RELACIONES CREADAS y NODOS "nuevos" cuando es necesario
#

EJEC. QRY [0]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Aluminium' AND b.name = 'Bauxite' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
EJEC. QRY [1]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Arsenic' AND b.name = 'Copper' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
EJEC. QRY [2]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Arsenic' AND b.name = 'Zinc' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
EJEC. QRY [3]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Cadmium' AND b.name = 'Zinc' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
EJEC. QRY [4]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Germanium' AND b.name = 'Zinc' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
EJEC. QRY [5]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Hafnium' AND b.name = 'Zirconium' CREATE (a)-[r :NECESITA_DE]->(b) RETURN type(r)
EJEC. QRY [6]> MATCH (a :MaterialB), (b  :MaterialB) WHERE a.name = 'Selenium' AND b.name = 'Copper' CREATE (a

# Fin proceso - RELACIONES CREADAS ENTRE MATERIALES ADICIONALES-Creados también los NODOS de materiales nuevos al estudio con prefijo "N-" cuando así está marcado en el fichero de datos- MaterialOrg con ese prefijo. 

In [8]:
# fin archivo